<h1 align="center">Extrair Vértices de Memorial Descritivo</h1>

O objetivo deste _notebook_ é demonstrar a extração de coordenadas geodésicas referente a um memorial descritivo da Unidade de Manejo Florestal 4, localizada na Floresta Nacional de Amapá. O arquivo está disponível neste [link](https://www.gov.br/agricultura/pt-br/assuntos/servico-florestal-brasileiro/concessao-florestal/concessoes-florestais-em-andamento-1/ContratoConcesso012021.pdf).

Utilizaremos o módulo `pdfminer.high_level` do pacote `pdfminer`.

In [1]:
# Carregar o módulo extract_text do pacote pdfminer
from pdfminer.high_level import extract_text

### Ler como texto texto apenas as páginas 26, 27 e 28 do arquivo pdf 

In [3]:
# Ler páginas 26, 27 e 28
pdf_txt = extract_text('./data/ContratoConcesso012021_umf1_flonas_amapa.pdf', page_numbers=[26, 27, 28])

### Mostrar o conteúdo das páginas selecionadas

In [10]:
# Mostrar o conteúdo das páginas
print(pdf_txt)

Unidade de Manejo Florestal (UMF) IV 

Área Plana: 39.356,25 ha 

Perímetro: 125.469,24 m 

Municípios: Ferreira Gomes/AP 

O limite da UMF IV foi definido seguindo orientações da equipe da Gerência Executiva 
de  Monitoramento  e  Auditoria  Florestal  (GEMAF),  utilizando  como  referência  a 
hidrografia  da  Base  Continua  1:250.000  (Bc250)  versão  2017,  da  Coordenação  de 
Cartografia da Diretoria de Geociências (DGC) do IBGE, Gerência de Bases Contínuas 
(GBC)  e  ainda  utilizando  imagens  de  Modelo  digital  de  Superfície  SRTM  de  30m  da 
USGS.  O  Perímetro  forma  um  polígono  irregular  de  22  vértices.  Inicia-se  a  descrição 
deste perímetro no vértice   P-01, de coordenadas  N 140.443,12m e  E 410.769,20m, 
localizado  na  margem  esquerda  do  Rio  Santo  Antônio;  daí,  segue  a  montante  pelos 
meandros do igarapé sem nome por aproximadamente e distância de 4.991,2m até o ponto   
P-02, de coordenadas   N 139.245,91m e  E 415.407,42m, localizado na marge

### Capturando Padrões
Utilizaremos expressões regulares (regex) para extração das informações de interesse, a saber: Código de cada vértice, coordenadas geodésicas UTM referente ao leste e norte.
Ao observamos o memorial descritivo acima percebemos que há um padrão definido para os códicos dos vértices, cada um inicia com a letra maiúscula `P` seguida de um hífen e de um número com dois dígitos, por exemplo o primeiro vértice é denominado como `P-01`, o segundo com `P-02` o terceiro como `P-03` e assim por diante até o vértice `P-29`.
A expressão regular a seguir captura todos os vértices disponíveis no memorial descritivo:

In [11]:
# importar o pacote "re" para usar com regex
import re

# Regex para extração dos nomes dos vértices
vert = re.findall("[A-Z]+-\d{2}", pdf_txt)
print(vert)

['P-01', 'P-02', 'P-03', 'P-04', 'P-05', 'P-06', 'P-07', 'P-08', 'P-09', 'P-10', 'P-11', 'P-12', 'P-13', 'P-14', 'P-15', 'P-17', 'P-18', 'P-19', 'P-20', 'P-21', 'P-01', 'P-23', 'P-24', 'P-25', 'P-26', 'P-27', 'P-28', 'P-29', 'P-23']


Vemos acima que a expressão não conseguiu extrair o nome do vértice 16, mas sabemos que os códigos são uma sequência de 1 até 29, podendo serem gerados apartir do seguinte código:

In [47]:
vert = []

for i in range(1, 10):
    vert.append('P-0'+str(i))
    
for i in range(10, 30):
    vert.append('P-'+str(i))

In [48]:
print(vert)
print()
print(len(vert))

['P-01', 'P-02', 'P-03', 'P-04', 'P-05', 'P-06', 'P-07', 'P-08', 'P-09', 'P-10', 'P-11', 'P-12', 'P-13', 'P-14', 'P-15', 'P-16', 'P-17', 'P-18', 'P-19', 'P-20', 'P-21', 'P-22', 'P-23', 'P-24', 'P-25', 'P-26', 'P-27', 'P-28', 'P-29']

29


Agora resta capturar o padrão das coordenadas UTM Leste e Norte. Pelo fato da área do memorial descritvo está localizada no hemisfério norte, mais precisamente no município de Ferreira Gomes, esatado do Amapá, ambas coordenadas apresentam-se com o mesmo número de dígitos. Em termos de padrão textual a coordenada norte sempre inicia com a letra maiúscula "N", 3 dígitos seguidos de um ponto, mais 3 dígitos, uma vígula e por fim dois dígitos e a letra minúscula "m", a exemplo da coordenada norte referente ao vértice P-29, N 118.764,82m. A coordenada leste possui o mesmo padrão com a diferença que sempre inicia com a letra maiúscula E, E 409.323,92m. De posse de tal padrão podemos aplicar uma expressão regular para extrair estas coordenadas.

In [42]:
norte = re.findall(r'N\s*?(\d{3}\.\d{3},\d{2})m\b', pdf_txt)
print(norte)
print(len(norte))

['140.443,12', '139.245,91', '137.260,28', '138.132,31', '130.563,09', '131.013,35', '130.950,90', '130.796,93', '126.064,02', '125.490,99', '123.506,24', '124.133,18', '122.731,73', '122.740,51', '122.286,80', '114.544,32', '111.841,13', '109.808,42', '115.818,01', '117.729,48', '120.389,80', '120.672,98', '119.583,44', '119.194,17', '117.796,60', '117.196,36', '117.364,55', '117.996,66', '118.764,82']
29


In [43]:
leste = re.findall(r'E\s*?(\d{3}\.\d{3},\d{2})m\b', pdf_txt)
print(leste)
print(len(leste))

['410.769,20', '415.407,42', '416.613,41', '418.765,81', '423.378,13', '423.560,87', '428.236,94', '428.294,17', '426.838,05', '424.926,37', '423.091,73', '422.056,57', '419.643,04', '417.505,46', '416.776,58', '411.181,91', '412.251,32', '412.385,57', '399.342,22', '400.652,90', '400.773,23', '400.212,53', '410.879,79', '411.683,91', '411.459,00', '410.688,84', '409.757,15', '409.403,14', '409.323,92']
29


### Gerar um dataframe com os dados e salvar como .csv
Agora que temos lista dos vértices, coordenadas leste e norte, podemos juntá-las em um _dataframe_ e salvá-lo no formato `.csv` ou `.xls`.

In [49]:
# Importar a biblioteca pandas
import pandas as pd

# gerar o dataframe
df = pd.DataFrame({
    'vertice': vert,
    'leste': leste,
    'norte': norte
})

In [57]:
# Salvar o data frame no disco nos formatos .csv e .xlsx
df.to_csv('./vertices_test.csv', index=False, sep=';')
df.to_excel('./vertices_test.xlsx', index=False)